In [19]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import tqdm.notebook as tq
from smogn import smoter

from common_stuff import PictureDataset
from common_stuff import device, transform

In [11]:
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),  # 16 x 112 x 112
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),  # 32 x 56 x 56
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),  # 64 x 28 x 28
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),  # 128 x 14 x 14
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),  # 256 x 7 x 7
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),  # 128 x 14 x 14
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),  # 64 x 28 x 28
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),  # 32 x 56 x 56
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),  # 16 x 112 x 112
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 3, stride=2, padding=1, output_padding=1),  # 3 x 224 x 224
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return "aboba", x


class TargetAwareAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super(TargetAwareAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_dim),
        )
        self.fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        bottleneck = self.encoder(x)
        reconstruction = self.decoder(bottleneck)
        target_pred = self.fc(reconstruction)
        return reconstruction, target_pred

In [3]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

df = pd.read_csv("../../post2ctr_dataset.csv")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = PictureDataset(train_df, transform=transform)
test_dataset = PictureDataset(test_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [14]:
conv_autoencoder = ConvAutoencoder()
conv_autoencoder = conv_autoencoder.to(device)
optimizer = optim.AdamW(conv_autoencoder.parameters(), lr=1e-4)
best_loss = float("inf")
epochs = 30
criterion = nn.MSELoss()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    running_loss = 0.0
    conv_autoencoder.train()
    loop = tq.tqdm(enumerate(train_loader), total=len(train_loader), leave=True, colour="steelblue")
    for batch_idx, data in loop:
        images = data["img"].to(device, dtype=torch.float).squeeze()
        targets = data["target"].to(device, dtype=torch.float)
                
        unnecessary, outputs = conv_autoencoder(images)
        outputs = outputs.squeeze()
        loss = criterion(outputs, images)

        running_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(conv_autoencoder.parameters(), max_norm=1.0)
        optimizer.step()
    running_loss /= len(train_loader)
    train_loss = running_loss

    running_loss = 0.0
    conv_autoencoder.eval()
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader, 0):
            images = data["img"].to(device, dtype=torch.float).squeeze()
            targets = data["target"].to(device, dtype=torch.float)
                    
            unnecessary, outputs = conv_autoencoder(images)
            outputs = outputs.squeeze()
            loss = criterion(outputs, images)
            running_loss += loss.item()
    running_loss /= len(test_loader)
    test_loss = running_loss

    print(f"Train MSE = {train_loss:.4f}, Test MSE = {test_loss:.4f}")
    if test_loss < best_loss:
        torch.save(conv_autoencoder.state_dict(), "../../ae_features.pth")

Epoch 1/30


  0%|          | 0/1177 [00:00<?, ?it/s]

Train MSE = 1.3166, Test MSE = 1.2044
Epoch 2/30


  0%|          | 0/1177 [00:00<?, ?it/s]

Train MSE = 1.1830, Test MSE = 1.1687
Epoch 3/30


...

Train MSE = 1.1026, Test MSE = 1.1074
Epoch 29/30


  0%|          | 0/1177 [00:00<?, ?it/s]

Train MSE = 1.1034, Test MSE = 1.0993
Epoch 30/30


  0%|          | 0/1177 [00:00<?, ?it/s]

Train MSE = 1.1049, Test MSE = 1.0925


In [9]:
conv_autoencoder = ConvAutoencoder()
conv_autoencoder.load_state_dict(torch.load("../../ae_features.pth"))
conv_autoencoder = conv_autoencoder.to(device)
bottleneck_features_train = []
bottleneck_targets_train = []
bottleneck_features_test = []
bottleneck_targets_test = []

conv_autoencoder.eval()
with torch.no_grad():
    for data in train_loader:
        images = data["img"].to(device)
        targets = data["target"].to(device)
        bottleneck = conv_autoencoder.encoder(images).view(images.size(0), -1).to(device)
        bottleneck_features_train.append(bottleneck)
        bottleneck_targets_train.append(targets)

with torch.no_grad():
    for data in test_loader:
        images = data["img"].to(device)
        targets = data["target"].to(device)
        bottleneck = conv_autoencoder.encoder(images).view(images.size(0), -1).to(device)
        bottleneck_features_test.append(bottleneck)
        bottleneck_targets_test.append(targets)

bottleneck_features_train = torch.cat(bottleneck_features_train, dim=0)
bottleneck_targets_train = torch.cat(bottleneck_targets_train, dim=0)
bottleneck_features_test = torch.cat(bottleneck_features_test, dim=0)
bottleneck_targets_test = torch.cat(bottleneck_targets_test, dim=0)

/tmp/ipykernel_740/3061250597.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  conv_autoencoder.load_state_dict(torch.load("../../ae_features.pth"))


In [13]:
input_dim = bottleneck_features_train.size(1)
target_aware_autoencoder = TargetAwareAutoencoder(input_dim)
target_aware_autoencoder = target_aware_autoencoder.to(device)
optimizer = optim.AdamW(target_aware_autoencoder.parameters(), lr=1e-4)
best_loss = float("inf")
epochs = 15
criterion = nn.MSELoss()

class BottleneckDataset(torch.utils.data.Dataset):
    def __init__(self, images, targets):
        self.images = images
        self.targets = targets

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]

        return {
            "img" : image,
            "target" : target
        }

train_dataset = BottleneckDataset(bottleneck_features_train, bottleneck_targets_train)
test_dataset = BottleneckDataset(bottleneck_features_test, bottleneck_targets_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    running_loss = 0.0
    target_aware_autoencoder.train()
    loop = tq.tqdm(enumerate(train_loader), total=len(train_loader), leave=True, colour="steelblue")
    for batch_idx, data in loop:
        images = data["img"].to(device, dtype=torch.float).squeeze()
        targets = data["target"].to(device, dtype=torch.float).squeeze()
                
        unnecessary, preds = target_aware_autoencoder(images)
        preds = preds.squeeze()
        loss = criterion(preds, targets)

        running_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(target_aware_autoencoder.parameters(), max_norm=1.0)
        optimizer.step()
    running_loss /= len(train_loader)
    train_loss = running_loss

    running_loss = 0.0
    target_aware_autoencoder.eval()
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader, 0):
            images = data["img"].to(device, dtype=torch.float).squeeze()
            targets = data["target"].to(device, dtype=torch.float).squeeze()
                    
            unnecessary, preds = target_aware_autoencoder(images)
            preds = preds.squeeze()
            loss = criterion(preds, targets)
            running_loss += loss.item()
    running_loss /= len(test_loader)
    test_loss = running_loss

    print(f"Train MSE = {train_loss:.4f}, Test MSE = {test_loss:.4f}")
    if test_loss < best_loss:
        torch.save(target_aware_autoencoder.state_dict(), "../../ae_target.pth")

Epoch 1/15


  0%|          | 0/1177 [00:00<?, ?it/s]

Train MSE = 0.0072, Test MSE = 0.0030
Epoch 2/15


  0%|          | 0/1177 [00:00<?, ?it/s]

Train MSE = 0.0033, Test MSE = 0.0032
Epoch 3/15


...

Train MSE = 0.0032, Test MSE = 0.0030
Epoch 14/15


  0%|          | 0/1177 [00:00<?, ?it/s]

Train MSE = 0.0032, Test MSE = 0.0030
Epoch 15/15


  0%|          | 0/1177 [00:00<?, ?it/s]

Train MSE = 0.0032, Test MSE = 0.0032


In [23]:
target_aware_autoencoder = TargetAwareAutoencoder(input_dim)
target_aware_autoencoder.load_state_dict(torch.load("../../ae_target.pth"))
target_aware_autoencoder = target_aware_autoencoder.to(device)
bottleneck2_features = []
bottleneck2_targets = []

print(bottleneck_features_train)

target_aware_autoencoder.eval()
with torch.no_grad():
    for data in train_loader:
        images = data["img"].to(device)
        targets = data["target"].to(device)
        bottleneck2 = target_aware_autoencoder.encoder(images).view(images.size(0), -1).to(device)
        
        bottleneck2_features.append(bottleneck2)
        bottleneck2_targets.append(targets)
        
with torch.no_grad():
    for data in test_loader:
        images = data["img"].to(device)
        targets = data["target"].to(device)
        bottleneck2 = target_aware_autoencoder.encoder(images).view(images.size(0), -1).to(device)
        
        bottleneck2_features.append(bottleneck2)
        bottleneck2_targets.append(targets)

bottleneck2_features = torch.cat(bottleneck2_features, dim=0)
bottleneck2_targets = torch.cat(bottleneck2_targets, dim=0)

/tmp/ipykernel_740/1409710821.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  target_aware_autoencoder.load_state_dict(torch.load("../../ae_target.pth"))


tensor([[12.5789, 15.9801, 15.9784,  ..., 16.6163, 11.0083, 13.7528],
        [24.8015, 28.2587, 26.2591,  ..., 21.6372, 19.1268, 17.5075],
        [16.4868, 17.6376, 17.5571,  ...,  8.1516,  8.3466,  8.1959],
        ...,
        [19.2720, 22.2409, 17.6407,  ..., 21.5559, 16.7291, 20.0648],
        [14.6280, 18.1209, 17.4168,  ...,  9.7090, 12.9908,  9.2360],
        [16.8705, 28.1130, 18.7760,  ..., 19.5477, 20.7211, 20.6541]],
       device='cuda:0')


In [29]:
def apply_oversampling(bottleneck_df):
    updated = smoter(data=bottleneck_df, y="target")
    return updated

bottleneck_features = torch.cat((bottleneck_features_train, bottleneck_features_test))
bottleneck_targets = torch.cat((bottleneck_targets_train, bottleneck_targets_test))
bottleneck_df = pd.DataFrame(bottleneck_features.cpu()).iloc[:, :128]
bottleneck_df["target"] = bottleneck_targets.cpu()
bottleneck_new = apply_oversampling(bottleneck_df)

dist_matrix:   0%|          | 3/4655 [00:08<3:34:25,  2.77s/it]


KeyboardInterrupt: 